# Domain Adaptation (領域適應)

**對應課程**: 李宏毅 2021 ML HW11 - Domain Adaptation

本 notebook 介紹如何處理訓練與測試資料分佈不同的問題：
- **Domain Shift 問題**: 理解分佈偏移
- **Discrepancy-based**: MMD, CORAL
- **Adversarial-based**: DANN, ADDA
- **Self-training**: Pseudo-labeling

```
Domain Adaptation 問題設定：

┌─────────────────────────────────────────────────────────────┐
│  Source Domain (有標籤)        Target Domain (無/少標籤)    │
│                                                             │
│  ┌─────────┐                   ┌─────────┐                  │
│  │ 🖼️ 🖼️ 🖼️ │                   │ 📷 📷 📷 │                  │
│  │ 合成圖片 │      ──────►     │ 真實照片 │                  │
│  │ (有標籤) │                   │ (無標籤) │                  │
│  └─────────┘                   └─────────┘                  │
│                                                             │
│  P_s(X, Y) ≠ P_t(X, Y)                                     │
│                                                             │
│  目標：學習一個在 Target Domain 表現良好的模型              │
└─────────────────────────────────────────────────────────────┘

Domain Adaptation 類型：
- Unsupervised DA: Target 完全無標籤
- Semi-supervised DA: Target 有少量標籤
- Supervised DA: Target 有標籤（但量少）
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, List, Dict, Tuple
from dataclasses import dataclass
from torch.autograd import Function

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 1: Domain Shift 視覺化

理解什麼是 Domain Shift 以及它對模型的影響。

In [ ]:
def create_domain_shift_data(
    n_samples: int = 500,
    source_params: Tuple[float, float] = (0, 1),
    target_params: Tuple[float, float] = (2, 1.5)
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    創建具有 domain shift 的二分類數據
    
    Returns:
        X_source, y_source, X_target, y_target
    """
    np.random.seed(42)
    
    # Source domain
    source_class0 = np.random.randn(n_samples // 2, 2) * source_params[1] + [source_params[0], 0]
    source_class1 = np.random.randn(n_samples // 2, 2) * source_params[1] + [source_params[0], 3]
    X_source = np.vstack([source_class0, source_class1])
    y_source = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
    
    # Target domain (shifted)
    target_class0 = np.random.randn(n_samples // 2, 2) * target_params[1] + [target_params[0], 0]
    target_class1 = np.random.randn(n_samples // 2, 2) * target_params[1] + [target_params[0], 3]
    X_target = np.vstack([target_class0, target_class1])
    y_target = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
    
    return X_source, y_source, X_target, y_target


def visualize_domain_shift():
    """視覺化 domain shift"""
    X_s, y_s, X_t, y_t = create_domain_shift_data()
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Source domain
    axes[0].scatter(X_s[y_s==0, 0], X_s[y_s==0, 1], c='blue', alpha=0.5, label='Class 0')
    axes[0].scatter(X_s[y_s==1, 0], X_s[y_s==1, 1], c='red', alpha=0.5, label='Class 1')
    axes[0].set_title('Source Domain (有標籤)')
    axes[0].legend()
    axes[0].set_xlim(-5, 8)
    axes[0].set_ylim(-4, 8)
    
    # Target domain
    axes[1].scatter(X_t[y_t==0, 0], X_t[y_t==0, 1], c='cyan', alpha=0.5, label='Class 0')
    axes[1].scatter(X_t[y_t==1, 0], X_t[y_t==1, 1], c='orange', alpha=0.5, label='Class 1')
    axes[1].set_title('Target Domain (無標籤)')
    axes[1].legend()
    axes[1].set_xlim(-5, 8)
    axes[1].set_ylim(-4, 8)
    
    # Both domains
    axes[2].scatter(X_s[:, 0], X_s[:, 1], c='blue', alpha=0.3, label='Source', marker='o')
    axes[2].scatter(X_t[:, 0], X_t[:, 1], c='red', alpha=0.3, label='Target', marker='x')
    axes[2].set_title('Domain Shift 視覺化')
    axes[2].legend()
    axes[2].set_xlim(-5, 8)
    axes[2].set_ylim(-4, 8)
    
    # 添加箭頭表示 shift
    axes[2].annotate('', xy=(2, 1.5), xytext=(0, 1.5),
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))
    axes[2].text(1, 2.2, 'Domain\nShift', ha='center', fontsize=10, color='green')
    
    plt.tight_layout()
    plt.show()

visualize_domain_shift()

In [ ]:
def demonstrate_performance_drop():
    """展示 domain shift 對模型性能的影響"""
    X_s, y_s, X_t, y_t = create_domain_shift_data()
    
    # 轉換為 tensor
    X_s_t = torch.FloatTensor(X_s)
    y_s_t = torch.LongTensor(y_s)
    X_t_t = torch.FloatTensor(X_t)
    y_t_t = torch.LongTensor(y_t)
    
    # 簡單的分類器
    class SimpleClassifier(nn.Module):
        def __init__(self):
            super().__init__()
            self.net = nn.Sequential(
                nn.Linear(2, 32),
                nn.ReLU(),
                nn.Linear(32, 32),
                nn.ReLU(),
                nn.Linear(32, 2)
            )
            
        def forward(self, x):
            return self.net(x)
    
    # 在 source 上訓練
    model = SimpleClassifier()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(100):
        optimizer.zero_grad()
        output = model(X_s_t)
        loss = criterion(output, y_s_t)
        loss.backward()
        optimizer.step()
    
    # 評估
    model.eval()
    with torch.no_grad():
        source_pred = model(X_s_t).argmax(dim=1)
        target_pred = model(X_t_t).argmax(dim=1)
        
        source_acc = (source_pred == y_s_t).float().mean().item()
        target_acc = (target_pred == y_t_t).float().mean().item()
    
    print(f"Source Domain 準確率: {source_acc:.2%}")
    print(f"Target Domain 準確率: {target_acc:.2%}")
    print(f"性能下降: {source_acc - target_acc:.2%}")
    
    return model, X_s, y_s, X_t, y_t

model, X_s, y_s, X_t, y_t = demonstrate_performance_drop()

## Part 2: Discrepancy-based Methods

透過最小化領域間的分佈差異來進行適應。

```
Discrepancy-based 方法原理：

┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│   Source Features ──┬──► 特徵分佈 P_s(f)                       │
│                     │                                           │
│                     │    ┌──────────────┐                      │
│                     ├───►│  Discrepancy │ ◄── 最小化            │
│                     │    │  (MMD/CORAL) │                      │
│                     │    └──────────────┘                      │
│                     │                                           │
│   Target Features ──┴──► 特徵分佈 P_t(f)                       │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘

MMD (Maximum Mean Discrepancy):
MMD²(P, Q) = E[k(x,x')] + E[k(y,y')] - 2E[k(x,y)]

其中 k 是核函數（通常用 RBF kernel）
```

In [ ]:
class MMDLoss(nn.Module):
    """Maximum Mean Discrepancy Loss"""
    
    def __init__(self, kernel_type: str = 'rbf', kernel_mul: float = 2.0, kernel_num: int = 5):
        super().__init__()
        self.kernel_type = kernel_type
        self.kernel_mul = kernel_mul
        self.kernel_num = kernel_num
        
    def gaussian_kernel(self, source: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """計算高斯核矩陣"""
        n_samples = source.shape[0] + target.shape[0]
        total = torch.cat([source, target], dim=0)
        
        # 計算 L2 距離
        total0 = total.unsqueeze(0).expand(n_samples, n_samples, -1)
        total1 = total.unsqueeze(1).expand(n_samples, n_samples, -1)
        L2_distance = ((total0 - total1) ** 2).sum(dim=2)
        
        # 多尺度高斯核
        bandwidth = torch.sum(L2_distance) / (n_samples ** 2 - n_samples)
        bandwidth /= self.kernel_mul ** (self.kernel_num // 2)
        bandwidth_list = [bandwidth * (self.kernel_mul ** i) for i in range(self.kernel_num)]
        
        kernel_val = torch.zeros_like(L2_distance)
        for bw in bandwidth_list:
            kernel_val += torch.exp(-L2_distance / (2 * bw + 1e-8))
            
        return kernel_val
    
    def forward(self, source: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        計算 MMD 損失
        
        Args:
            source: Source domain 特徵 [batch_s, dim]
            target: Target domain 特徵 [batch_t, dim]
        """
        batch_s = source.shape[0]
        batch_t = target.shape[0]
        
        kernels = self.gaussian_kernel(source, target)
        
        # 分割核矩陣
        XX = kernels[:batch_s, :batch_s]
        YY = kernels[batch_s:, batch_s:]
        XY = kernels[:batch_s, batch_s:]
        YX = kernels[batch_s:, :batch_s]
        
        # MMD
        mmd = XX.mean() + YY.mean() - XY.mean() - YX.mean()
        
        return mmd


# 測試
mmd_loss = MMDLoss()

source_features = torch.randn(32, 64)
target_features = torch.randn(32, 64) + 1  # 有偏移

loss = mmd_loss(source_features, target_features)
print(f"MMD Loss: {loss.item():.4f}")

# 相同分佈
same_features = torch.randn(32, 64)
loss_same = mmd_loss(same_features, same_features + torch.randn(32, 64) * 0.1)
print(f"MMD Loss (相近分佈): {loss_same.item():.4f}")

In [ ]:
class CORALLoss(nn.Module):
    """Correlation Alignment Loss"""
    
    def forward(self, source: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """
        計算 CORAL 損失
        對齊 source 和 target 的協方差矩陣
        
        Args:
            source: Source domain 特徵 [batch_s, dim]
            target: Target domain 特徵 [batch_t, dim]
        """
        d = source.shape[1]
        
        # 中心化
        source_centered = source - source.mean(dim=0, keepdim=True)
        target_centered = target - target.mean(dim=0, keepdim=True)
        
        # 計算協方差矩陣
        cov_source = (source_centered.T @ source_centered) / (source.shape[0] - 1)
        cov_target = (target_centered.T @ target_centered) / (target.shape[0] - 1)
        
        # CORAL loss: Frobenius norm of covariance difference
        loss = torch.sum((cov_source - cov_target) ** 2) / (4 * d * d)
        
        return loss


# 測試
coral_loss = CORALLoss()
loss = coral_loss(source_features, target_features)
print(f"CORAL Loss: {loss.item():.4f}")

In [ ]:
class DeepCORAL(nn.Module):
    """Deep CORAL 網路"""
    
    def __init__(
        self,
        input_dim: int = 2,
        hidden_dim: int = 64,
        feature_dim: int = 32,
        num_classes: int = 2
    ):
        super().__init__()
        
        # 特徵提取器
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, feature_dim)
        )
        
        # 分類器
        self.classifier = nn.Linear(feature_dim, num_classes)
        
        # CORAL loss
        self.coral_loss = CORALLoss()
        
    def forward(
        self,
        source: torch.Tensor,
        target: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor, Optional[torch.Tensor]]:
        """
        Args:
            source: Source domain 數據
            target: Target domain 數據（訓練時使用）
            
        Returns:
            source_output: Source 分類輸出
            source_features: Source 特徵
            coral_loss: CORAL 損失（如果有 target）
        """
        # 提取特徵
        source_features = self.feature_extractor(source)
        source_output = self.classifier(source_features)
        
        coral = None
        if target is not None:
            target_features = self.feature_extractor(target)
            coral = self.coral_loss(source_features, target_features)
            
        return source_output, source_features, coral


# 測試
model = DeepCORAL(input_dim=2)
source = torch.randn(32, 2)
target = torch.randn(32, 2)

output, features, coral = model(source, target)
print(f"輸出形狀: {output.shape}")
print(f"特徵形狀: {features.shape}")
print(f"CORAL Loss: {coral.item():.4f}")

## Part 3: Adversarial-based Methods

使用對抗學習讓特徵提取器產生領域不變的特徵。

```
DANN (Domain-Adversarial Neural Network) 架構：

┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│   Input X ──► Feature Extractor (G_f) ──► Features f            │
│                                              │                  │
│                                    ┌────────┴────────┐          │
│                                    │                 │          │
│                                    ▼                 ▼          │
│                          ┌─────────────────┐  ┌──────────────┐  │
│                          │ Label Predictor │  │   Domain     │  │
│                          │     (G_y)       │  │ Discriminator│  │
│                          │                 │  │    (G_d)     │  │
│                          └────────┬────────┘  └──────┬───────┘  │
│                                   │                  │          │
│                                   ▼                  ▼          │
│                           Class Labels         Domain Label     │
│                                                (Source/Target)  │
│                                                                 │
│   關鍵：Gradient Reversal Layer (GRL)                           │
│   - 前向傳播：恆等函數                                          │
│   - 反向傳播：梯度反轉 (× -λ)                                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
class GradientReversalFunction(Function):
    """Gradient Reversal Layer 的自動微分函數"""
    
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.view_as(x)
    
    @staticmethod
    def backward(ctx, grad_output):
        # 反向傳播時反轉梯度
        return grad_output.neg() * ctx.lambda_, None


class GradientReversalLayer(nn.Module):
    """Gradient Reversal Layer"""
    
    def __init__(self, lambda_: float = 1.0):
        super().__init__()
        self.lambda_ = lambda_
        
    def forward(self, x):
        return GradientReversalFunction.apply(x, self.lambda_)
    
    def set_lambda(self, lambda_: float):
        self.lambda_ = lambda_


class DANN(nn.Module):
    """Domain-Adversarial Neural Network"""
    
    def __init__(
        self,
        input_dim: int = 2,
        hidden_dim: int = 64,
        feature_dim: int = 32,
        num_classes: int = 2
    ):
        super().__init__()
        
        # 特徵提取器 G_f
        self.feature_extractor = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_dim, feature_dim),
            nn.ReLU()
        )
        
        # 標籤預測器 G_y
        self.label_predictor = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        # 領域判別器 G_d
        self.domain_discriminator = nn.Sequential(
            nn.Linear(feature_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 2)  # Source vs Target
        )
        
        # Gradient Reversal Layer
        self.grl = GradientReversalLayer()
        
    def forward(
        self,
        x: torch.Tensor,
        lambda_: float = 1.0
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        Args:
            x: 輸入數據
            lambda_: GRL 的 lambda 參數
            
        Returns:
            class_output: 類別預測
            domain_output: 領域預測
            features: 特徵
        """
        self.grl.set_lambda(lambda_)
        
        # 提取特徵
        features = self.feature_extractor(x)
        
        # 類別預測
        class_output = self.label_predictor(features)
        
        # 領域預測（通過 GRL）
        reversed_features = self.grl(features)
        domain_output = self.domain_discriminator(reversed_features)
        
        return class_output, domain_output, features


# 測試
dann = DANN()
x = torch.randn(32, 2)
class_out, domain_out, feat = dann(x, lambda_=1.0)
print(f"類別輸出: {class_out.shape}")
print(f"領域輸出: {domain_out.shape}")
print(f"特徵: {feat.shape}")

In [ ]:
def train_dann(
    model: DANN,
    source_data: Tuple[torch.Tensor, torch.Tensor],
    target_data: torch.Tensor,
    num_epochs: int = 100,
    lr: float = 0.01
) -> List[Dict[str, float]]:
    """
    訓練 DANN
    
    Args:
        model: DANN 模型
        source_data: (X_source, y_source)
        target_data: X_target
        num_epochs: 訓練輪數
        lr: 學習率
        
    Returns:
        訓練歷史
    """
    X_s, y_s = source_data
    X_t = target_data
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    class_criterion = nn.CrossEntropyLoss()
    domain_criterion = nn.CrossEntropyLoss()
    
    # 領域標籤
    source_domain_label = torch.zeros(X_s.shape[0]).long()  # 0 = source
    target_domain_label = torch.ones(X_t.shape[0]).long()   # 1 = target
    
    history = []
    
    for epoch in range(num_epochs):
        model.train()
        
        # 計算 lambda（逐漸增加）
        p = epoch / num_epochs
        lambda_ = 2 / (1 + np.exp(-10 * p)) - 1
        
        optimizer.zero_grad()
        
        # Source domain
        class_out_s, domain_out_s, _ = model(X_s, lambda_)
        class_loss = class_criterion(class_out_s, y_s)
        domain_loss_s = domain_criterion(domain_out_s, source_domain_label)
        
        # Target domain
        _, domain_out_t, _ = model(X_t, lambda_)
        domain_loss_t = domain_criterion(domain_out_t, target_domain_label)
        
        # 總損失
        domain_loss = (domain_loss_s + domain_loss_t) / 2
        total_loss = class_loss + domain_loss
        
        total_loss.backward()
        optimizer.step()
        
        history.append({
            'epoch': epoch,
            'class_loss': class_loss.item(),
            'domain_loss': domain_loss.item(),
            'lambda': lambda_
        })
        
        if (epoch + 1) % 20 == 0:
            print(f"Epoch {epoch+1}: class_loss={class_loss.item():.4f}, "
                  f"domain_loss={domain_loss.item():.4f}, λ={lambda_:.4f}")
            
    return history


# 訓練 DANN
X_s, y_s, X_t, y_t = create_domain_shift_data()
X_s_t = torch.FloatTensor(X_s)
y_s_t = torch.LongTensor(y_s)
X_t_t = torch.FloatTensor(X_t)
y_t_t = torch.LongTensor(y_t)

dann_model = DANN(input_dim=2)
history = train_dann(dann_model, (X_s_t, y_s_t), X_t_t, num_epochs=100)

In [ ]:
def evaluate_dann(model, X_s, y_s, X_t, y_t):
    """評估 DANN 模型"""
    model.eval()
    
    with torch.no_grad():
        X_s_t = torch.FloatTensor(X_s)
        X_t_t = torch.FloatTensor(X_t)
        
        class_out_s, _, feat_s = model(X_s_t)
        class_out_t, _, feat_t = model(X_t_t)
        
        source_pred = class_out_s.argmax(dim=1).numpy()
        target_pred = class_out_t.argmax(dim=1).numpy()
        
        source_acc = (source_pred == y_s).mean()
        target_acc = (target_pred == y_t).mean()
        
    print(f"\nDANN 結果:")
    print(f"Source Domain 準確率: {source_acc:.2%}")
    print(f"Target Domain 準確率: {target_acc:.2%}")
    
    # 視覺化特徵空間
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    feat_s_np = feat_s.numpy()
    feat_t_np = feat_t.numpy()
    
    # 使用 PCA 降維到 2D（如果特徵維度 > 2）
    if feat_s_np.shape[1] > 2:
        from sklearn.decomposition import PCA
        pca = PCA(n_components=2)
        all_feat = np.vstack([feat_s_np, feat_t_np])
        all_feat_2d = pca.fit_transform(all_feat)
        feat_s_2d = all_feat_2d[:len(feat_s_np)]
        feat_t_2d = all_feat_2d[len(feat_s_np):]
    else:
        feat_s_2d = feat_s_np
        feat_t_2d = feat_t_np
    
    # 按領域著色
    axes[0].scatter(feat_s_2d[:, 0], feat_s_2d[:, 1], c='blue', alpha=0.5, label='Source')
    axes[0].scatter(feat_t_2d[:, 0], feat_t_2d[:, 1], c='red', alpha=0.5, label='Target')
    axes[0].set_title('特徵空間（按領域著色）')
    axes[0].legend()
    
    # 按類別著色
    colors_s = ['blue' if y == 0 else 'cyan' for y in y_s]
    colors_t = ['red' if y == 0 else 'orange' for y in y_t]
    axes[1].scatter(feat_s_2d[:, 0], feat_s_2d[:, 1], c=colors_s, alpha=0.5, marker='o')
    axes[1].scatter(feat_t_2d[:, 0], feat_t_2d[:, 1], c=colors_t, alpha=0.5, marker='x')
    axes[1].set_title('特徵空間（按類別著色）')
    
    plt.tight_layout()
    plt.show()

evaluate_dann(dann_model, X_s, y_s, X_t, y_t)

## Part 4: Self-Training (Pseudo-Labeling)

使用模型預測的偽標籤來進行自訓練。

```
Self-Training 流程：

┌─────────────────────────────────────────────────────────────────┐
│                                                                 │
│  Step 1: 在 Source 上訓練初始模型                               │
│          Model_0 ← train(Source_labeled)                        │
│                                                                 │
│  Step 2: 對 Target 生成偽標籤                                   │
│          pseudo_labels ← Model_t(Target_unlabeled)              │
│          選擇高置信度樣本                                       │
│                                                                 │
│  Step 3: 合併訓練                                               │
│          Model_{t+1} ← train(Source + Target_pseudo)            │
│                                                                 │
│  Step 4: 重複 Step 2-3 直到收斂                                 │
│                                                                 │
│  置信度閾值選擇：                                               │
│  ├─ 太高：利用的 target 樣本太少                               │
│  └─ 太低：引入太多錯誤標籤                                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

In [ ]:
class SelfTrainer:
    """Self-Training Domain Adaptation"""
    
    def __init__(
        self,
        model: nn.Module,
        confidence_threshold: float = 0.9,
        max_iterations: int = 10
    ):
        self.model = model
        self.confidence_threshold = confidence_threshold
        self.max_iterations = max_iterations
        
    def generate_pseudo_labels(
        self,
        X_target: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """
        生成偽標籤
        
        Returns:
            pseudo_labels: 偽標籤
            confidence: 置信度
            mask: 高置信度樣本遮罩
        """
        self.model.eval()
        with torch.no_grad():
            logits = self.model(X_target)
            probs = F.softmax(logits, dim=1)
            confidence, pseudo_labels = probs.max(dim=1)
            mask = confidence >= self.confidence_threshold
            
        return pseudo_labels, confidence, mask
    
    def train_step(
        self,
        X_source: torch.Tensor,
        y_source: torch.Tensor,
        X_target_pseudo: torch.Tensor,
        y_target_pseudo: torch.Tensor,
        optimizer: torch.optim.Optimizer,
        source_weight: float = 1.0,
        target_weight: float = 1.0
    ) -> float:
        """單步訓練"""
        self.model.train()
        criterion = nn.CrossEntropyLoss()
        
        optimizer.zero_grad()
        
        # Source loss
        source_out = self.model(X_source)
        source_loss = criterion(source_out, y_source)
        
        # Target pseudo loss
        if len(X_target_pseudo) > 0:
            target_out = self.model(X_target_pseudo)
            target_loss = criterion(target_out, y_target_pseudo)
            total_loss = source_weight * source_loss + target_weight * target_loss
        else:
            total_loss = source_loss
            
        total_loss.backward()
        optimizer.step()
        
        return total_loss.item()
    
    def fit(
        self,
        X_source: torch.Tensor,
        y_source: torch.Tensor,
        X_target: torch.Tensor,
        epochs_per_iteration: int = 50,
        lr: float = 0.01
    ) -> List[Dict]:
        """
        完整的 self-training 流程
        """
        optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        history = []
        
        for iteration in range(self.max_iterations):
            print(f"\nIteration {iteration + 1}/{self.max_iterations}")
            
            # 生成偽標籤
            pseudo_labels, confidence, mask = self.generate_pseudo_labels(X_target)
            
            num_selected = mask.sum().item()
            print(f"  選擇了 {num_selected}/{len(X_target)} 個高置信度樣本")
            
            if num_selected == 0:
                print("  沒有高置信度樣本，降低閾值或停止")
                break
                
            # 訓練
            X_target_pseudo = X_target[mask]
            y_target_pseudo = pseudo_labels[mask]
            
            for epoch in range(epochs_per_iteration):
                loss = self.train_step(
                    X_source, y_source,
                    X_target_pseudo, y_target_pseudo,
                    optimizer
                )
                
            print(f"  最終 Loss: {loss:.4f}")
            
            history.append({
                'iteration': iteration,
                'num_pseudo': num_selected,
                'loss': loss
            })
            
        return history


# 測試
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
        
    def forward(self, x):
        return self.net(x)

model_st = SimpleModel()
trainer = SelfTrainer(model_st, confidence_threshold=0.8, max_iterations=5)
history = trainer.fit(X_s_t, y_s_t, X_t_t, epochs_per_iteration=30)

In [ ]:
# 評估 Self-Training
model_st.eval()
with torch.no_grad():
    pred = model_st(X_t_t).argmax(dim=1).numpy()
    acc = (pred == y_t).mean()
    
print(f"Self-Training Target 準確率: {acc:.2%}")

## Part 5: 方法比較

In [ ]:
def print_comparison_table():
    """印出 Domain Adaptation 方法比較表"""
    print("""
╔═══════════════════════════════════════════════════════════════════════════════╗
║                     Domain Adaptation 方法比較                                ║
╠═══════════════════╦═══════════════════════════════════════════════════════════╣
║       方法        ║                      特點                                 ║
╠═══════════════════╬═══════════════════════════════════════════════════════════╣
║                   ║ - 最小化領域間的分佈差異（MMD, CORAL）                    ║
║  Discrepancy      ║ - 簡單直觀，易於實現                                      ║
║  -based           ║ - 不需要對抗訓練                                          ║
║                   ║ - 對於大的 domain shift 效果有限                          ║
╠═══════════════════╬═══════════════════════════════════════════════════════════╣
║                   ║ - 使用對抗學習產生領域不變特徵（DANN, ADDA）              ║
║  Adversarial      ║ - 理論基礎紮實（H-divergence）                            ║
║  -based           ║ - 效果通常較好                                            ║
║                   ║ - 訓練不穩定，需要仔細調參                                ║
╠═══════════════════╬═══════════════════════════════════════════════════════════╣
║                   ║ - 使用偽標籤進行自訓練                                    ║
║  Self-Training    ║ - 簡單，不需要修改網路架構                                ║
║                   ║ - 可能累積錯誤                                            ║
║                   ║ - 需要合適的置信度閾值                                    ║
╠═══════════════════╬═══════════════════════════════════════════════════════════╣
║                   ║ - 生成目標風格的數據                                      ║
║  Generation       ║ - CycleGAN, PixelDA                                       ║
║  -based           ║ - 需要額外的生成模型                                      ║
║                   ║ - 適用於視覺任務                                          ║
╚═══════════════════╩═══════════════════════════════════════════════════════════╝

選擇指南：
┌────────────────────────────────────────────────────────────────┐
│ 場景                              │ 推薦方法                   │
├────────────────────────────────────────────────────────────────┤
│ 小 domain shift                   │ Discrepancy-based (CORAL)  │
│ 中等 domain shift                 │ DANN / ADDA                │
│ 有少量目標標籤                    │ Self-Training + Fine-tune  │
│ 視覺任務（圖像風格差異）          │ Generation-based           │
│ 需要穩定訓練                      │ MMD + Self-Training        │
└────────────────────────────────────────────────────────────────┘
""")

print_comparison_table()

## 練習

### Exercise 1: 實作 ADDA (Adversarial Discriminative Domain Adaptation)

In [ ]:
class ADDA:
    """Adversarial Discriminative Domain Adaptation"""
    
    def __init__(
        self,
        source_encoder: nn.Module,
        target_encoder: nn.Module,  # 初始化為 source_encoder 的拷貝
        classifier: nn.Module,
        discriminator: nn.Module
    ):
        # TODO: 實作 ADDA
        # ADDA 與 DANN 的主要區別：
        # 1. 有兩個獨立的編碼器（source 和 target）
        # 2. 訓練分兩階段：
        #    - 先在 source 上訓練 source_encoder + classifier
        #    - 再固定 source_encoder，用對抗學習訓練 target_encoder
        pass
    
    def pretrain_source(self, X_source, y_source, epochs=100):
        """在 source 上預訓練"""
        pass
    
    def adapt_target(self, X_source, X_target, epochs=100):
        """對抗適應 target encoder"""
        pass

### Exercise 2: 實作 Domain-Specific Batch Normalization

In [ ]:
class DomainSpecificBatchNorm(nn.Module):
    """Domain-Specific Batch Normalization"""
    
    def __init__(self, num_features: int, num_domains: int = 2):
        super().__init__()
        # TODO: 實作 DSBN
        # 為每個 domain 維護獨立的 BN 統計量
        pass
    
    def forward(self, x: torch.Tensor, domain_idx: int) -> torch.Tensor:
        # 根據 domain_idx 選擇對應的 BN
        pass

### Exercise 3: 比較不同方法在不同 domain shift 程度下的表現

In [ ]:
def compare_methods_across_shifts():
    """
    比較不同 DA 方法在不同 domain shift 程度下的表現
    
    TODO:
    1. 創建不同 shift 程度的數據（shift = [0.5, 1.0, 2.0, 3.0]）
    2. 訓練各種方法：Baseline, CORAL, DANN, Self-Training
    3. 記錄 target 準確率
    4. 繪製比較圖
    """
    pass

## 總結

```
Domain Adaptation 重點回顧：

1. 問題定義
   ├─ Source Domain: 有標籤
   ├─ Target Domain: 無/少標籤
   └─ 目標: 在 Target 上表現良好

2. Discrepancy-based
   ├─ MMD: 最小化均值差異
   ├─ CORAL: 對齊協方差矩陣
   └─ 簡單但對大 shift 效果有限

3. Adversarial-based
   ├─ DANN: GRL + Domain Discriminator
   ├─ ADDA: 兩階段訓練
   └─ 效果好但訓練不穩定

4. Self-Training
   ├─ 偽標籤迭代訓練
   ├─ 置信度閾值選擇重要
   └─ 可能累積錯誤

實際應用建議：
- 先嘗試簡單方法（CORAL, Self-Training）
- 效果不佳再考慮 DANN
- 結合多種方法通常效果更好
```